<a href="https://colab.research.google.com/github/Next-Sunshine/TFF0318/blob/master/TestForTFFCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#环境测试
#@test {"skip":true}
!pip install --quiet --upgrade tensorflow_federated
%load_ext tensorboard

     |████████████████████████████████| 430kB 7.1MB/s 
     |████████████████████████████████| 2.8MB 23.1MB/s 
     |████████████████████████████████| 296kB 41.2MB/s 
     |████████████████████████████████| 421.8MB 42kB/s 
     |████████████████████████████████| 2.2MB 38.6MB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 20.0MB 161kB/s 
     |████████████████████████████████| 450kB 42.4MB/s 
     |████████████████████████████████| 3.9MB 31.0MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [5]:
import collections

import numpy as np
import random
import tensorflow as tf
import tensorflow_federated as tff
import h5py

tf.compat.v1.enable_v2_behavior()

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, world!')()

b'Hello, world!'

In [6]:
#装载数据集，实验使用EMNIST数据集
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
#装载mnist数据集
mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
#从mnist_train中读60000张图片放入一个数组中以备用
class_ = [[]for i in range(10)]
for i in range(len(mnist_train[1])):
  label = mnist_train[1][i]
  class_[label].append(mnist_train[0][i])

In [0]:
#当创建数据集需要修改的时候就要用这里，先清空再重新创建
f.flush()
f.clear()

In [0]:
#设置各个不平衡的数据集，首先弄一个BAL1:10个类别，每个类别数目相等，每个用户都含有10个类别且每类数目相等，均12个
#创建自己的数据集首先是BAL1：全平衡，使用了mnist数据集来分配，60000张图片平均分给500个客户端，每个客户端分得10类。每类120张图片
f = h5py.File("BAL1.hdf5","a")
top_group = f.create_group("examples")   #创建顶层group名为examples

In [0]:
MY_NUM_CLIENTS = 20  #写的时候最初用50，用500时间太长，试过500，磁盘会从30G飙升到60G且还运行不完，不知道后面500个客户端的时候怎么办
NUM_PER_CLASS = 12  #每个客户端每类的图片数量
#Collections.Orderdict->TensorSliceDataset+client_id->hdf5->HDF5ClientData
#<TensorSliceDataset shapes: OrderedDict([(label, ()), (pixels, (28, 28))]), types: OrderedDict([(label, tf.int32), (pixels, tf.float32)])>
pixels = []
label=[]
client_ids = []
for i in range(MY_NUM_CLIENTS):  #i客户端数量
  for j in range(10):  #j是类别数
    for k in range(NUM_PER_CLASS): #k控制每一类图片放多少张在客户端
      pixels.append(class_[j][i*NUM_PER_CLASS+k]/255.0) #转成0-1之内的数字再加入pixels，class_[j]存放的是第j类数
      label.append(j)  #加入label
  
  client_ids.append("f00_" + str(i))
  temp_group = top_group.create_group(name="f00_" + str(i))  #创建以client_id为名字的group
  label_ds = temp_group.create_dataset(name="label", data=np.array(label, dtype='int32'))  #向group中写入label
  pixels_ds = temp_group.create_dataset(name="pixels", data=np.array(pixels, dtype='float32'))  #向group中写入pixels

  #清空上一个pixels数组和label数组中存放的上一个客户端的数据信息,否则会有灾难！！！
  pixels.clear()
  label.clear()


In [0]:
#实例化构造的数据集
BAL1 = tff.simulation.hdf5_client_data.HDF5ClientData("BAL1.hdf5")
num_clients_BAL1 = len(BAL1.client_ids)
#留出总客户端的4分之一做测试集
BAL1_train, BAL1_test = BAL1.train_test_client_split(client_data=BAL1, num_test_clients= num_clients_BAL1//4)

In [71]:
print(len(BAL1_train.client_ids))

15


In [69]:
#BAL1.client_ids
#这里面的每一个打印出来都是和example_dataset类型一样的

x = tf.data.Dataset.from_tensor_slices(
        collections.OrderedDict((name, ds.value) for name, ds in sorted(
            f["examples"][BAL1.client_ids[0]].items())))
print(x)
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0]
)
print(example_dataset)

<TensorSliceDataset shapes: OrderedDict([(label, ()), (pixels, (28, 28))]), types: OrderedDict([(label, tf.int32), (pixels, tf.float32)])>
<TensorSliceDataset shapes: OrderedDict([(label, ()), (pixels, (28, 28))]), types: OrderedDict([(label, tf.int32), (pixels, tf.float32)])>


In [13]:
#此处得到客户端的总数K，因为每一轮随机选择C×K的向上取整
#根据FedAvg做的研究C取0.2可获得较好的性能，为了简化模型K这里取固定的数
K = len(emnist_train.client_ids)
C = 0.2
K

3383

以上是产生数据集的部分，以下是预处理以及其他部分

In [0]:
#NUM_CLIENTS在实验的时候应该是K×C
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

#预处理，将图片中的‘pixels'、‘label'分别表示成x和y
#将28×28的图像展平成784个元素，打乱顺序
def preprocess(dataset):
  #内部函数将像素和标签转换成x和y，并将像素展平
  def batch_format_fn(element):
    return collections.OrderedDict(
        #因为是灰度图所以通道数为1，如果是rgb图片则为3
        #因为CNN卷积层接收一个4d向量
        x = tf.reshape(element['pixels'], [-1,28,28,1]),
        y = tf.reshape(element['label'], [-1,1])
    )
  
  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [0]:
#为指定用户创建联邦数据,接收训练集和用户id
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
    ]


In [49]:
#这里改成随机获得NUM_CLIENTS个用户，模拟联邦学习每轮的随机选择用户
# sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
# federated_train_data = make_federated_data(emnist_train, sample_clients)
example_dataset = BAL1.create_tf_dataset_for_client(
    BAL1.client_ids[0]
)
print("examle_dataset",example_dataset)
# sample_clients = emnist_train.client_ids[0:3]
# federated_train_data = make_federated_data(emnist_train, sample_clients)

#BAL1数据集测试
#sample_clients = random.sample(BAL1.client_ids, NUM_CLIENTS)
sample_clients = BAL1.client_ids[0:NUM_CLIENTS]
federated_train_data = make_federated_data(BAL1, sample_clients)

examle_dataset <TensorSliceDataset shapes: OrderedDict([(label, ()), (pixels, (28, 28))]), types: OrderedDict([(label, tf.int32), (pixels, tf.float32)])>


间隔界

In [17]:
'''
#！自己添加代码中，正在想办法把训练模型改成CNN
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷积层神经元（卷积核）数目
            kernel_size=[5, 5],     # 感受野大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        #print("kernel:",kernel_size)
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
 
    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)
        #tf.nn.softmax()将原始输出归一化，且能凸显原始向量中最大的值                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output
        '''


'\n#！自己添加代码中，正在想办法把训练模型改成CNN\nclass CNN(tf.keras.Model):\n    def __init__(self):\n        super().__init__()\n        self.conv1 = tf.keras.layers.Conv2D(\n            filters=32,             # 卷积层神经元（卷积核）数目\n            kernel_size=[5, 5],     # 感受野大小\n            padding=\'same\',         # padding策略（vaild 或 same）\n            activation=tf.nn.relu   # 激活函数\n        )\n        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)\n        self.conv2 = tf.keras.layers.Conv2D(\n            filters=64,\n            kernel_size=[5, 5],\n            padding=\'same\',\n            activation=tf.nn.relu\n        )\n        #print("kernel:",kernel_size)\n        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)\n        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))\n        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)\n        self.dense2 = tf.keras.layers.Dense(units=10)\n \n    def call(self, inputs):\n        

间隔界


In [0]:
#创建一个变量集合来表示所有变量,包括model(weights+bias)以及metrics(num_examples,loss_sum,accuracy_sum)
MnistVariables = collections.namedtuple(
    'MnistVariables','cnn_conv2d_kernel cnn_conv2d_bias cnn_conv2d_1_kernel cnn_conv2d_1_bias cnn_dense_kernel cnn_dense_bias cnn_dense_1_kernel cnn_dense_1_bias num_examples loss_sum accuracy_sum'
)

In [0]:
#创建变量并初始化
def create_mnist_variables():
  #核初始化函数，使用正态分布，先使用tf.XXX(初始参数)生成一个模型关键字C，再使用C(arg1)传参数进去
  kernel_initalizer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05)
  return MnistVariables(
      #第一层卷积在5×5的patch中算出32个特征，权重张量形状为[5,5,1,32]
      #前两个维度是patch的大小，接着是输入的通道数，最后是输出的通道数目
      #h_conv1.shape=[-1,28,28,32]，第一个pooling层将其变成[-1,14,14,32]     
      cnn_conv2d_kernel=tf.Variable(
          #权重初始化至关重要，不能全部初始化为0！！！
          initial_value=kernel_initalizer(shape=(5,5,1,32), dtype=tf.float32),
          name='cnn_conv2d_kernel',
          trainable=True),
      #32个输出通道，每个输出通道都有一个对应的偏置
      cnn_conv2d_bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(32,)),
          name='cnn_conv2d_bias',
          trainable=True),
      #第二层卷积,64个过滤器，共享权重矩阵为32*5*5,h_conv2.shape=[-1,14,14,64]
      #第二个pooling层将其变成[-1,7,7,64]
      cnn_conv2d_1_kernel=tf.Variable(
          initial_value=kernel_initalizer(shape=(5,5,32,64), dtype=tf.float32),
          name='cnn_conv2d_1_kernel',
          trainable=True),
      #64个输出通道对应64个偏置
      cnn_conv2d_1_bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(64,)),
          name='cnn_conv2d_1_bias',
          trainable=True),
      #全连接层，现在图片尺寸减小到7×7，加入一个有1024个神经元的全连接层用于处理整个图片
      #将池化层输出的张量reshape成一些7*7*64的向量，乘上权重加上偏置然后使用ReLU
      cnn_dense_kernel=tf.Variable(
          initial_value=kernel_initalizer(shape=(7*7*64,1024), dtype=tf.float32),
          name='cnn_dense_kernel',
          trainable=True),
      cnn_dense_bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(1024,)),
          name='cnn_dense_bias',
          trainable=True),
      #这里以后可以加一个Dropout
      cnn_dense_1_kernel=tf.Variable(
          initial_value=kernel_initalizer(shape=(1024,10), dtype=tf.float32),
          name='cnn_dense_1_kernel',
          trainable=True),
      cnn_dense_1_bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(10,)),
          name='cnn_dense_1_bias',
          trainable=True),
      num_examples=tf.Variable(0.0, name='num_examples', trainable=False),
      loss_sum=tf.Variable(0.0, name='loss_sum', trainable=False),
      accuracy_sum=tf.Variable(0.0, name='accuracy_sum', trainable=False)
  )

In [0]:
#自定义前向传播函数
def mnist_forward_pass(variables, batch):
  _input_ = batch['x']  #取训练图片

  #神经网络
  #第一层卷积
  conv = tf.nn.conv2d(_input_, variables.cnn_conv2d_kernel, strides=[1,1,1,1], padding='SAME')                  # [batch_size, 28, 28, 32]
  pre_activation = tf.nn.bias_add(conv, variables.cnn_conv2d_bias)
  conv1 = tf.nn.relu(pre_activation, name='conv1')

  #第一层卷积层的池化
  pool1 = tf.nn.max_pool2d(conv1, ksize=[2, 2], strides=2, padding='SAME')

  #第二层卷积
  conv = tf.nn.conv2d(pool1, variables.cnn_conv2d_1_kernel, strides=[1,1,1,1], padding='SAME')
  pre_activation = tf.nn.bias_add(conv, variables.cnn_conv2d_1_bias)
  conv2 = tf.nn.relu(pre_activation, name='conv2')

  #第二层卷积的池化
  pool2 = tf.nn.max_pool2d(conv2, ksize=[2,2], strides=2, padding='SAME')
  print("pool2",pool2)
  
  #flatten
  flatten = tf.reshape(pool2, shape=(-1,7*7*64))
  print("flatten over")

  #全连接层1
  dense1 = tf.nn.relu(tf.matmul(flatten, variables.cnn_dense_kernel) + variables.cnn_dense_bias)
  print("dense1 over")

  #添加一个Dropout防止过拟合
  #keep_drop = tf.placeholder(tf.float32)
  #drop_dense1 = tf.nn.tropout(dense1, variables.keep_drop)

  #全连接层2
  dense2 = tf.matmul(dense1, variables.cnn_dense_1_kernel) + variables.cnn_dense_1_bias
  print("dense2 over")

  y_pred = tf.nn.softmax(dense2)
  print("y_pred",y_pred)

  #计算损失
  flat_labels = tf.reshape(batch['y'], [-1])
  loss = -tf.reduce_mean(
      tf.reduce_sum(tf.one_hot(flat_labels, 10) * tf.math.log(y_pred), axis=[1]))
  
  
  #计算准确率
  predictions = tf.cast(tf.argmax(y_pred,1), tf.int32)
  accuracy = tf.reduce_mean(
      tf.cast(tf.equal(predictions, flat_labels), tf.float32))
  
  #样本数
  num_examples = tf.cast(tf.size(batch['y']), tf.float32)

  #更新样本数、损失和、精度和,每一批都考虑了自己的权重
  variables.num_examples.assign_add(num_examples)
  variables.loss_sum.assign_add(loss * num_examples)
  variables.accuracy_sum.assign_add(accuracy * num_examples)

  
  return loss, predictions

In [0]:
#计算本地用户的metrics度量
def get_local_mnist_metrics(variables):
  return collections.OrderedDict(
      num_examples=variables.num_examples,
      loss=variables.loss_sum / variables.num_examples,
      accuracy=variables.accuracy_sum / variables.num_examples
  )

In [0]:
#集合每个设备发出的本地度量
@tff.federated_computation
def aggregate_mnist_metrics_across_clients(metrics):
  return collections.OrderedDict(
      num_examples=tff.federated_sum(metrics.num_examples),
      loss=tff.federated_mean(metrics.loss, metrics.num_examples),
      accuracy=tff.federated_mean(metrics.accuracy, metrics.num_examples)
  )

In [0]:
#自定义模型，创建tff.learning.model实例
class MnistModel(tff.learning.Model):

  def __init__(self):
    self._variables = create_mnist_variables()

  #所有的“tf.Variables”都应该在“__init__”中引入
  @property
  def trainable_variables(self):
    #return [self._variables.weights, self._variables.bias]
    return [self._variables.cnn_conv2d_kernel,
        self._variables.cnn_conv2d_bias,
        self._variables.cnn_conv2d_1_kernel,
        self._variables.cnn_conv2d_1_bias,
        self._variables.cnn_dense_kernel,
        self._variables.cnn_dense_bias,
        self._variables.cnn_dense_1_kernel,
        self._variables.cnn_dense_1_bias
        ]
  
  @property
  def non_trainable_variables(self):
    return []
  
  @property
  def local_variables(self):
    return [self._variables.num_examples, self._variables.loss_sum,
         self._variables.accuracy_sum]
  
  @property
  def input_spec(self):
    return collections.OrderedDict(
        x=tf.TensorSpec([None, 28,28,1], tf.float32),
        y=tf.TensorSpec([None, 1], tf.int32)
    )

  @tf.function
  def forward_pass(self, batch, training=True):
    del training
    loss, predictions = mnist_forward_pass(self._variables, batch)
    #取出样本数
    num_examples = tf.shape(batch['x'])[0]
    return tff.learning.BatchOutput(
        loss=loss, predictions=predictions, num_examples=num_examples)
    
  @tf.function
  def report_local_outputs(self):
    return get_local_mnist_metrics(self._variables)
  
  @property
  def federated_output_computation(self):
    return aggregate_mnist_metrics_across_clients

In [24]:
#创建迭代器执行联合平均的迭代过程
iterative_process = tff.learning.build_federated_averaging_process(
    MnistModel,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02)
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


pool2 Tensor("MaxPool2d_1:0", shape=(None, 7, 7, 64), dtype=float32)
flatten over
dense1 over
dense2 over
y_pred Tensor("Softmax:0", shape=(None, 10), dtype=float32)


In [0]:
#获得初始状态
state = iterative_process.initialize()

In [67]:
#查看第一轮训练
# sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
# federated_train_data = make_federated_data(emnist_train, sample_clients)
#BAL1数据集测试
# sample_clients = random.sample(BAL1.client_ids, NUM_CLIENTS)
# federated_train_data = make_federated_data(BAL1, sample_clients)

sample_clients = BAL1.client_ids[0:NUM_CLIENTS]
federated_train_data = make_federated_data(BAL1, sample_clients)

state, metrics = iterative_process.next(state, federated_train_data)
print('round 1, metrics={}'.format(metrics))

round 1, metrics=<num_examples=6000.0,loss=2.066589593887329,accuracy=0.39649999141693115>


In [75]:
#计算更多轮
NUM_ROUNDS = 30
for round_num in range(2,NUM_ROUNDS):
  #随机选择NUM_CLIENTS个用户
  # sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
  # federated_train_data = make_federated_data(emnist_train, sample_clients)

  sample_clients = random.sample(BAL1_train.client_ids, NUM_CLIENTS)
  federated_train_data = make_federated_data(BAL1_train, sample_clients)

  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  2, metrics=<num_examples=6000.0,loss=2.0423457622528076,accuracy=0.38499999046325684>
round  3, metrics=<num_examples=6000.0,loss=1.2473881244659424,accuracy=0.643833339214325>
round  4, metrics=<num_examples=6000.0,loss=0.6714187860488892,accuracy=0.8021666407585144>
round  5, metrics=<num_examples=6000.0,loss=0.4409526288509369,accuracy=0.8756666779518127>
round  6, metrics=<num_examples=6000.0,loss=0.3223763108253479,accuracy=0.9086666703224182>
round  7, metrics=<num_examples=6000.0,loss=0.24849900603294373,accuracy=0.9356666803359985>
round  8, metrics=<num_examples=6000.0,loss=0.2506083548069,accuracy=0.9315000176429749>
round  9, metrics=<num_examples=6000.0,loss=0.17909502983093262,accuracy=0.9526666402816772>
round 10, metrics=<num_examples=6000.0,loss=0.1639392226934433,accuracy=0.9585000276565552>
round 11, metrics=<num_examples=6000.0,loss=0.17241646349430084,accuracy=0.953000009059906>
round 12, metrics=<num_examples=6000.0,loss=0.14724187552928925,accuracy=0.961000

In [0]:
'''
#使用tensorboard可视化
#使用Tensorboard可视化这些联邦计算的度量
#创建目录和相应的摘要编写器
#@test {"skip": true}
logdir = "/tmp/logs/scalars/training/"
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()

#!!!太坑了，我以为@test是无关紧要的东西，emmm，就省略了，没想到就凉凉
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    #随机选择用户
    sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
    federated_train_data = make_federated_data(emnist_train, sample_clients)

    state, metrics = iterative_process.next(state, federated_train_data)

    print('round {:2d}, metrics={}'.format(round_num, metrics))

    for name, value in metrics._asdict().items():
      tf.summary.scalar(name, value, step=round_num)
'''

round  1, metrics=<num_examples=4520.0,loss=2.2766871452331543,accuracy=0.1285398304462433>
round  2, metrics=<num_examples=4755.0,loss=2.258615493774414,accuracy=0.18107254803180695>
round  3, metrics=<num_examples=4835.0,loss=2.224494218826294,accuracy=0.242192342877388>
round  4, metrics=<num_examples=5030.0,loss=2.1682469844818115,accuracy=0.3049701750278473>
round  5, metrics=<num_examples=5260.0,loss=2.093106508255005,accuracy=0.3969581723213196>
round  6, metrics=<num_examples=4775.0,loss=1.9683749675750732,accuracy=0.43287956714630127>
round  7, metrics=<num_examples=4890.0,loss=1.7429050207138062,accuracy=0.5300613641738892>
round  8, metrics=<num_examples=4750.0,loss=1.5358659029006958,accuracy=0.5734736919403076>
round  9, metrics=<num_examples=5310.0,loss=1.1962789297103882,accuracy=0.6866289973258972>
round 10, metrics=<num_examples=5055.0,loss=0.94955974817276,accuracy=0.7507418394088745>
round 11, metrics=<num_examples=5035.0,loss=0.7275433540344238,accuracy=0.8077457547

In [0]:
'''
#这个tensorBoard像有猫病一样，昨天还可以今天来突然就不行了
#@test {"skip":true}
%load_ext tensorboard
%tensorboard --logdir /tmp/logs/scalars/ --port=0
'''

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2020-04-06 08:24:12.865933: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-04-06 08:24:12.866088: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-04-06 08:24:12.866105: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, 

In [76]:
#在测试集上进行测试
#求值传递MnistModel就足够了，不执行梯度下降，也不需要构造优化器，MnistTModel就是前面定义的那个class
evaluation = tff.learning.build_federated_evaluation(MnistModel)

pool2 Tensor("MaxPool2d_1:0", shape=(None, 7, 7, 64), dtype=float32)
flatten over
dense1 over
dense2 over
y_pred Tensor("Softmax:0", shape=(None, 10), dtype=float32)


In [0]:
#评估一下在训练中得到的最新状态，为了从服务器状态中提取最新的训练模型，只需访问.model成员
#和上面说的形式一样，它接收model和联邦数据并返回训练的metrics
#这里是在评估模型的最新状态，为了从服务器状态中提取最新的训练模型，只需访问.model成员
train_metrics = evaluation(state.model, federated_train_data)

In [79]:
#编译联邦数据的测试样本，并对测试数据重新运行求值。同样也是随机选择NUM_CLIENTS个用户。
# sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
# federated_test_data = make_federated_data(emnist_test, sample_clients)

#BAL1测试精度
sample_clients = random.sample(BAL1_test.client_ids, 5)
federated_test_data = make_federated_data(BAL1_test, sample_clients)
len(federated_test_data), federated_test_data[0]

(5,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>)

In [80]:
#使用EMNIST，客户端随机选择10个，初始精度在0.8898
#使用BAL1，客户端随机选了5个作为测试集测试，精度在0.9616666436195374
test_metrics = evaluation(state.model, federated_test_data)

str(test_metrics)

'<num_examples=3000.0,loss=0.14176583290100098,accuracy=0.9616666436195374>'